In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer


# Gather Data 
- The dataset that you will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user @dog_rates, also known as WeRateDogs. WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog.

- Twitter archive enchaned.csv Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything.

- image predictions a table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction

- twitter Api this additional data can be gathered by anyone from Twitter's API.
----

In [6]:
df_twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
df_image_prediction = pd.read_csv('image-predictions.tsv', sep='\t')

In [115]:
# read twitter api from text contain json data
twitter_api= []
with open('tweet_json.txt') as file:
    for line in file:
        tweet = json.loads(line)
        try:
            tweet_id = tweet['id']
            retweet_count = tweet['retweet_count']
            favourites_count = tweet['favorite_count']
            followers_count = tweet['user']['followers_count']
            created_at = tweet['user']['created_at']
            twitter_api.append({"tweet_id":tweet_id,
                               "retweet_count":retweet_count,
                               "favourites_count":favourites_count,
                               "followers_count":followers_count,
                                "created_at":created_at
                               })
        except KeyError:
            print("Columns not here")

In [116]:
df_json = pd.DataFrame(twitter_api,columns=['tweet_id','followers_count','favourites_count','retweet_count','created_at'])

## Make A Copy OF Data
- Three New Data Frame Contain a copy of the orignal Data

In [117]:
clean_twitter_archive = df_twitter_archive.copy()
clean_image_prediction = df_image_prediction.copy()
clean_json= df_json.copy()

In [60]:
# This method return information about clean_twitter_archive
clean_twitter_archive.head(20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [36]:
clean_twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [135]:
# denominator should be 10
clean_twitter_archive['rating_denominator'].value_counts().sort_values(ascending=False)

10     2333
11        3
50        3
80        2
20        2
7         1
2         1
16        1
40        1
0         1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
70        1
Name: rating_denominator, dtype: int64

In [136]:
#should be from 0 to 10 sometimes people will give higher value than 10 
clean_twitter_archive['rating_numerator'].value_counts().sort_values(ascending=False)

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
121       1
80        1
204       1
24        1
143       1
99        1
45        1
27        1
17        1
1776      1
960       1
666       1
182       1
26        1
144       1
88        1
84        1
165       1
60        1
50        1
44        1
20        1
Name: rating_numerator, dtype: int64

In [138]:
clean_twitter_archive['name'].value_counts()

None          745
a              55
Charlie        12
Oliver         11
Lucy           11
Cooper         11
Lola           10
Penny          10
Tucker         10
Winston         9
Bo              9
Sadie           8
the             8
an              7
Bailey          7
Daisy           7
Buddy           7
Toby            7
Stanley         6
Bella           6
Koda            6
Jax             6
Scout           6
Milo            6
Dave            6
Leo             6
Jack            6
Rusty           6
Oscar           6
Oakley          5
             ... 
Batdog          1
Major           1
Stella          1
Steve           1
Stephanus       1
Philippe        1
Kollin          1
Longfellow      1
Jed             1
Dudley          1
Combo           1
Bluebert        1
Nida            1
Bruiser         1
Remus           1
Nigel           1
Brandi          1
Millie          1
Siba            1
Petrick         1
Brady           1
Chubbs          1
Tripp           1
Duchess         1
Sully     

# Assess And Clean Data
## Data Quality Problem In clean_twitter_archive

- Invalid data Type. (type of `timestamp and retweeted_status_timestamp` are object should be Date Type) 
- Inaccurate data : Like Nan and None are object data type Instead Of Null Type In all columns
- `rating_numerator has many extremly value` I'll replace IT to 20 as Extremely Value Or more lovely dog just for analysis
- `rating_denominator` should not exceed 10 I'll replace all values to 10
- `name` has many Missing and Invalid Name like None,an,a,the I'll replace it with `No Name`
- Missing Value In (```in_reply_to_status_id ,in_reply_to_user_id,retweeted_status_id,retweeted_status_timestamp,expanded_urls```)

## Messey OR Tidy Data In clean_twitter_archive
- kind of dogs(`doggo, floofer,pupper,puppo`)Should be in one column and each row should be an observation


In [149]:
clean_twitter_archive['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64